<a href="https://colab.research.google.com/github/misrori/goldhandtrading/blob/main/rsi_strategy_tester_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install goldhand

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=fa5ffa74509a4b16c3d3887f1efad96c65ac249c723d63a13bea5bf3f8eceb4f
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
from goldhand import *
from tqdm import tqdm
from IPython.display import display
import numpy as np
import pandas_ta as ta
import warnings
pd.set_option('display.max_rows', 70)
warnings.filterwarnings('ignore')

tw=Tw()



# RSI

In [3]:
def rsi_strategy(data, buy_threshold = 30, sell_threshold = 70):

    in_trade = False  # Flag to track if already in a trade
    trade_id = 1
    all_trades = []

    temp_trade = {}

    for i in range(1, len(data)):
        # Check if not already in a trade
        if not in_trade:
            # Generate buy signal
            #You have to change olne the buy and sell signal
            if data['rsi'][i] < buy_threshold:

                if i == (len(data) -1):
                    temp_trade['buy_price'] = data['close'][i]
                    temp_trade.update(dict(data.iloc[i].add_prefix('buy_')))
                else:
                    temp_trade['buy_price'] = data['open'][i+1]
                    temp_trade.update(dict(data.iloc[i+1].add_prefix('buy_')))

                temp_trade['trade_id'] = trade_id
                temp_trade['status'] = 'open'
                in_trade = True  # Set flag to indicate in a trade
        else:
            # Generate sell signal
            #You have to change olne the buy and sell signal
            if data['rsi'][i] > sell_threshold:
                if i == (len(data) -1):
                    temp_trade['sell_price'] = data['close'][i]
                    temp_trade.update(dict(data.iloc[i].add_prefix('sell_')))
                else:
                    temp_trade['sell_price'] = data['open'][i+1]
                    temp_trade.update(dict(data.iloc[i+1].add_prefix('sell_')))

                temp_trade['trade_id'] = trade_id
                temp_trade['status'] = 'closed'


                # calculate results
                temp_trade['result'] = temp_trade['sell_price'] / temp_trade['buy_price']
                temp_trade['days_in_trade'] = (temp_trade['sell_date'] - temp_trade['buy_date']).days

                in_trade = False  # Reset flag to indicate not in a trade
                trade_id +=1
                all_trades.append(temp_trade)
                temp_trade = {}

    if temp_trade:
        temp_trade['sell_price'] = data['close'][i]
        temp_trade['trade_id'] = trade_id
        temp_trade['sell_date'] = data['date'][i]

        temp_trade['result'] = temp_trade['sell_price'] / temp_trade['buy_price']
        temp_trade['days_in_trade'] = (temp_trade['sell_date'] - temp_trade['buy_date']).days
        all_trades.append(temp_trade)

    res_df = pd.DataFrame(all_trades)

    # change orders
    all_col = res_df.columns.tolist()
    first = ['result', 'buy_price', 'sell_price', 'buy_date', 'sell_date', 'days_in_trade']
    first.extend([x for x in all_col if x not in first])
    res_df = res_df[first]
    return(res_df)


In [4]:
ticker= 'TSLA'

data = GoldHand(ticker).df
display(data)


In [5]:
# backtest
backtest = Backtest( data, rsi_strategy, plot_title=tw.get_plotly_title(ticker),  buy_threshold=30, sell_threshold=70)


In [6]:
backtest.trades

,ticker,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,buy_open,buy_low,buy_high,...,sell_sma_100,sell_diff_sma100,sell_sma_200,sell_diff_sma200,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_local,sell_local_text
0,TSLA,1.187284,1.541333,1.830000,2011-01-21,2011-04-01,70,1.541333,1.514000,1.572667,...,1.777093,0.013486,NaN,NaN,1.430576,1.910491,7.491997,-19.509962,,
1,TSLA,1.302691,1.610000,2.097333,2011-08-09,2011-11-04,87,1.610000,1.580000,1.696667,...,1.771853,21.567627,1.735997,24.078582,1.799126,2.247007,4.317890,-16.475125,,
2,TSLA,1.309917,1.774667,2.324667,2012-01-17,2012-02-21,35,1.774667,1.760667,1.822667,...,1.970353,16.730332,1.874743,22.683459,2.187164,2.357369,2.494294,-4.905892,,
3,TSLA,1.483966,7.962000,11.815333,2013-11-19,2014-01-23,65,7.962000,7.948000,8.600000,...,10.556940,14.616549,8.735753,38.511241,11.114429,12.291438,1.582132,-8.145220,,
4,TSLA,1.121381,13.956000,15.650000,2014-12-09,2015-04-28,140,13.956000,13.618000,14.515333,...,13.789680,11.426318,15.126517,1.578790,14.114397,15.712269,2.257918,-8.141286,,
5,TSLA,1.022130,14.218667,14.533333,2015-10-13,2016-03-16,155,14.218667,14.075333,14.834667,...,13.797080,7.235248,15.272557,-3.124714,13.393092,15.089575,1.988747,-9.477591,,
6,TSLA,1.034171,13.852000,14.325333,2016-05-13,2016-12-27,228,13.852000,13.780000,14.080000,...,13.500547,8.405485,14.305750,2.303850,13.511036,14.697231,0.422933,-7.682073,,
7,TSLA,1.157629,19.966667,23.114000,2017-11-03,2018-06-13,222,19.966667,19.675333,20.416668,...,20.740253,10.824743,21.517787,6.820153,20.434942,23.655725,2.916608,-11.095732,,
8,TSLA,0.912318,18.217333,16.620001,2018-09-10,2019-07-16,309,18.217333,18.066668,19.068666,...,16.450200,2.280414,18.861390,-10.794843,15.531782,17.227685,2.391346,-7.688114,,
9,TSLA,2.136338,29.334000,62.667332,2020-03-17,2020-06-09,84,29.334000,26.400000,31.456667,...,46.636073,34.469585,33.884420,85.074244,55.773542,64.855523,3.419141,-11.063060,,


In [7]:
backtest.trades_summary

{'ticker': 'TSLA',
 'number_of_trades': 15,
 'win_ratio(%)': 80.0,
 'average_res(%)': 22.01,
 'average_trade_len(days)': 133.0,
 'median_res(%)': 16.85,
 'cumulative_result': 13.271689219144967,
 'trade_results': '18.73 # 30.27 # 30.99 # 48.4 # 12.14 # 2.21 # 3.42 # 15.76 # -8.77 # 113.63 # 37.16 # 16.85 # -20.49 # 30.88 # -1.05',
 'profitable_trade_results': '18.73 # 30.27 # 30.99 # 48.4 # 12.14 # 2.21 # 3.42 # 15.76 # 113.63 # 37.16 # 16.85 # 30.88',
 'profitable_trades_mean': 30.04,
 'profitable_trades_median': 24.5,
 'looser_trade_results': '-8.77 # -20.49 # -1.05',
 'looser_trades_mean': -10.1,
 'looser_trades_median': -8.77,
 'median_trade_len(days)': 112.0,
 'number_of_win_trades': 12,
 'number_of_lost_trades': 3,
 'max_gain(%)': 113.63,
 'max_lost(%)': -20.49,
 'first_trade_buy': datetime.date(2011, 1, 21),
 'first_data_date': datetime.date(2010, 6, 29),
 'first_open_price': 1.2666670083999634,
 'last_data_date': datetime.date(2024, 1, 18),
 'last_close_price': 211.880004882812

In [8]:
pd.DataFrame(backtest.trades_summary, index=['Trades summary']).T

,Trades summary
ticker,TSLA
number_of_trades,15
win_ratio(%),80.0
average_res(%),22.01
average_trade_len(days),133.0
median_res(%),16.85
cumulative_result,13.271689
trade_results,18.73 # 30.27 # 30.99 # 48.4 # 12.14 # 2.21 # ...
profitable_trade_results,18.73 # 30.27 # 30.99 # 48.4 # 12.14 # 2.21 # ...
profitable_trades_mean,30.04


In [9]:
backtest.show_trades()

In [ ]:
backtest.summarize_strategy()

,Strategy summary
ticker,TSLA
number_of_trades,15
win_ratio(%),86.67
average_res(%),22.25
average_trade_len(days),133.0
median_res(%),16.85
cumulative_result,13.764648
trade_results,18.73 # 30.27 # 30.99 # 48.4 # 12.14 # 2.21 # ...
profitable_trade_results,18.73 # 30.27 # 30.99 # 48.4 # 12.14 # 2.21 # ...
profitable_trades_mean,27.93


,ticker,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,buy_open,buy_low,buy_high,...,sell_sma_100,sell_diff_sma100,sell_sma_200,sell_diff_sma200,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_local,sell_local_text
0,TSLA,1.187284,1.541333,1.830000,2011-01-21,2011-04-01,70,1.541333,1.514000,1.572667,...,1.777093,0.013486,NaN,NaN,1.430576,1.910491,7.491997,-19.509962,,
1,TSLA,1.302691,1.610000,2.097333,2011-08-09,2011-11-04,87,1.610000,1.580000,1.696667,...,1.771853,21.567627,1.735997,24.078582,1.799126,2.247007,4.317890,-16.475125,,
2,TSLA,1.309917,1.774667,2.324667,2012-01-17,2012-02-21,35,1.774667,1.760667,1.822667,...,1.970353,16.730332,1.874743,22.683459,2.187164,2.357369,2.494294,-4.905892,,
3,TSLA,1.483966,7.962000,11.815333,2013-11-19,2014-01-23,65,7.962000,7.948000,8.600000,...,10.556940,14.616549,8.735753,38.511241,11.114429,12.291438,1.582132,-8.145220,,
4,TSLA,1.121381,13.956000,15.650000,2014-12-09,2015-04-28,140,13.956000,13.618000,14.515333,...,13.789680,11.426318,15.126517,1.578790,14.114397,15.712269,2.257918,-8.141286,,
5,TSLA,1.022130,14.218667,14.533333,2015-10-13,2016-03-16,155,14.218667,14.075333,14.834667,...,13.797080,7.235248,15.272557,-3.124714,13.393092,15.089575,1.988747,-9.477591,,
6,TSLA,1.034171,13.852000,14.325333,2016-05-13,2016-12-27,228,13.852000,13.780000,14.080000,...,13.500547,8.405485,14.305750,2.303850,13.511036,14.697231,0.422933,-7.682073,,
7,TSLA,1.157629,19.966667,23.114000,2017-11-03,2018-06-13,222,19.966667,19.675333,20.416668,...,20.740253,10.824743,21.517787,6.820153,20.434942,23.655725,2.916608,-11.095732,,
8,TSLA,0.912318,18.217333,16.620001,2018-09-10,2019-07-16,309,18.217333,18.066668,19.068666,...,16.450200,2.280414,18.861390,-10.794843,15.531782,17.227685,2.391346,-7.688114,,
9,TSLA,2.136338,29.334000,62.667332,2020-03-17,2020-06-09,84,29.334000,26.400000,31.456667,...,46.636073,34.469585,33.884420,85.074244,55.773542,64.855523,3.419141,-11.063060,,


In [10]:
show_indicator_rsi_strategy('TSLA', plot_title=tw.get_plotly_title('TSLA'),  buy_threshold = 25, sell_threshold = 80, add_strategy_summary = True)

# Apply for top 20 stocks




In [ ]:
all_trades = []
all_trades_summaries=[]

for ticker in tqdm(tw.stock['name'][0:20]):
    try:
        # get data
        data = GoldHand(ticker).df

        # backtest
        backtest = Backtest( data, rsi_strategy, plot_title=tw.get_plotly_title(ticker), buy_threshold=28, sell_threshold=75)

        # results of backtest
        trades = backtest.trades
        trades_summary= backtest.trades_summary

        # append results
        all_trades.append(trades)
        all_trades_summaries.append(trades_summary)

    except:
      pass


tradesdf = pd.concat(all_trades, ignore_index=True)
trades_summarydf = pd.DataFrame(all_trades_summaries)



100%|██████████| 20/20 [00:10<00:00,  1.90it/s]


In [ ]:
trades_summarydf

,ticker,number_of_trades,win_ratio(%),average_res(%),average_trade_len(days),median_res(%),cumulative_result,trade_results,profitable_trade_results,profitable_trades_mean,...,looser_trades_mean,looser_trades_median,median_trade_len(days),number_of_win_trades,number_of_lost_trades,max_gain(%),max_lost(%),first_trade_buy,buy_threshold,sell_threshold
0,AAPL,9,77.78,16.23,224.0,7.72,3.479083,36.53 # 31.91 # -1.58 # -11.45 # 7.72 # 30.13 ...,36.53 # 31.91 # 7.72 # 30.13 # 6.63 # 41.03 # ...,22.73,...,-6.51,-6.51,179.0,7,2,41.03,-11.45,2006-02-10,28,75
1,MSFT,9,77.78,7.82,242.0,14.39,1.825421,5.51 # 14.39 # -21.79 # -5.34 # 14.67 # 4.79 #...,5.51 # 14.39 # 14.67 # 4.79 # 15.39 # 23.4 # 19.4,13.94,...,-13.57,-13.57,170.0,7,2,23.40,-21.79,2006-05-01,28,75
2,GOOG,12,83.33,13.89,191.0,13.20,4.505082,19.98 # 6.74 # 5.18 # -1.7 # 10.02 # 23.01 # 2...,19.98 # 6.74 # 5.18 # 10.02 # 23.01 # 27.74 # ...,17.19,...,-2.62,-2.62,175.0,10,2,33.83,-3.54,2006-02-06,28,75
3,AMZN,8,87.50,21.29,335.0,16.77,3.516816,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,1.19 # 59.95 # 30.66 # 9.05 # 70.0 # 11.66 # 2...,29.20,...,-34.09,-34.09,262.5,7,1,70.00,-34.09,2006-02-03,28,75
4,NVDA,9,66.67,18.53,367.0,28.37,3.113139,66.11 # -39.26 # 29.65 # -14.24 # 1.59 # 34.55...,66.11 # 29.65 # 1.59 # 34.55 # 28.37 # 63.04,37.22,...,-18.83,-14.24,395.0,6,3,66.11,-39.26,2006-05-19,28,75
5,META,8,87.50,18.50,173.0,23.57,3.196840,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,16.67 # 39.95 # 20.2 # 10.95 # 26.94 # 35.37 #...,26.35,...,-36.41,-36.41,105.0,7,1,39.95,-36.41,2012-07-30,28,75
6,TSLA,11,90.91,40.08,180.0,18.14,23.948082,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,43.13 # 116.51 # 12.16 # 37.87 # 9.49 # 18.14 ...,44.81,...,-7.21,-7.21,134.0,10,1,154.55,-7.21,2012-01-17,28,75
7,LLY,12,91.67,11.19,217.0,11.99,3.173300,4.43 # -28.71 # 11.61 # 10.66 # 16.32 # 10.44 ...,4.43 # 11.61 # 10.66 # 16.32 # 10.44 # 12.37 #...,14.81,...,-28.71,-28.71,168.0,11,1,35.88,-28.71,2006-02-07,28,75
8,V,7,100.00,34.70,474.0,27.32,7.441021,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,81.51 # 34.59 # 17.0 # 27.32 # 14.64 # 24.02 #...,34.70,...,NaN,NaN,439.0,7,0,81.51,14.64,2008-10-08,28,75
9,JPM,11,90.91,10.71,303.0,9.13,2.948838,9.13 # -4.11 # 6.65 # 16.23 # 11.05 # 35.78 # ...,9.13 # 6.65 # 16.23 # 11.05 # 35.78 # 10.28 # ...,12.19,...,-4.11,-4.11,197.0,10,1,35.78,-4.11,2007-03-06,28,75


In [ ]:
tradesdf

,ticker,result,buy_price,sell_price,buy_date,sell_date,days_in_trade,buy_open,buy_low,buy_high,...,sell_sma_100,sell_diff_sma100,sell_sma_200,sell_diff_sma200,sell_bb_lower,sell_bb_upper,sell_diff_upper_bb,sell_diff_lower_bb,sell_local,sell_local_text
0,AAPL,1.365296,2.327857,3.178214,2006-02-10,2006-11-22,285,2.327857,2.246429,2.416786,...,2.552104,26.380336,2.430057,32.727623,2.991023,3.249548,0.750016,-7.265357,,
1,AAPL,1.319094,4.999643,6.595000,2008-01-24,2008-05-06,103,4.999643,4.714643,5.025000,...,5.428157,22.812013,5.498979,21.230315,6.159912,6.786946,1.807812,-7.598031,,
2,AAPL,0.984237,4.780714,4.705357,2008-09-16,2009-05-05,231,4.780714,4.719643,5.089286,...,3.561411,33.083306,4.053750,16.919965,4.365946,4.819626,1.687520,-7.884491,,
3,AAPL,0.885468,20.022499,17.729286,2012-11-08,2013-08-15,280,20.022499,19.117500,20.079643,...,15.554496,14.323851,16.760509,6.097610,15.938828,18.458173,3.799653,-10.367900,,
4,AAPL,1.077202,28.237499,30.417500,2015-08-05,2017-01-26,540,28.237499,28.025000,29.360001,...,28.479700,7.041158,26.786475,13.807437,29.729285,30.657715,0.566556,-2.478975,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,COST,1.067977,219.339996,234.250000,2018-10-08,2019-03-13,156,219.339996,219.259995,224.500000,...,218.331400,8.266610,219.291650,7.792524,215.439543,242.016463,2.384490,-8.858813,,
183,COST,1.161051,294.440002,341.859985,2020-03-02,2020-08-05,156,294.440002,291.380005,309.510010,...,307.750700,10.469286,304.925800,11.492698,318.477478,345.294532,1.566177,-6.321888,,
184,COST,1.234062,335.510010,414.040009,2021-02-26,2021-07-12,136,335.510010,330.940002,335.890015,...,366.998300,11.139481,367.703050,10.926468,397.334554,415.041435,1.755769,-2.585430,,
185,COST,1.237788,489.890015,606.380005,2022-01-19,2022-04-08,79,489.890015,487.100006,496.239990,...,535.099899,12.136066,492.778599,21.766647,562.110862,615.221120,2.530022,-6.321098,,
